In [1]:
import os

In [2]:
pwd

'd:\\college_work\\4th year\\Sem7th\\Project\\OBE\\Paper- Blooms taxanomy\\Boolm-s-Level-Detection-A-MLOPS-Project\\research'

In [3]:
os.chdir("../")
%pwd

'd:\\college_work\\4th year\\Sem7th\\Project\\OBE\\Paper- Blooms taxanomy\\Boolm-s-Level-Detection-A-MLOPS-Project'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainingConfig:
    root_dir: Path
    model_name: str
    train_data_path: Path 
    test_data_path: Path

In [5]:
from blooms.constants import *
from blooms.utils.common import read_yaml,create_directories

class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH,
        schema_filepath=SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_model_training_config(self) -> ModelTrainingConfig:
        config = self.config.model_training 
        
        create_directories([config.root_dir])
        
        model_training_config = ModelTrainingConfig(
            root_dir=config.root_dir,
            model_name=config.model_name,
            train_data_path=config.train_data_path,
            test_data_path=config.test_data_path
        )
        
        return model_training_config